In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

np.set_printoptions(suppress=True,threshold=None)

## Read OBJ
vertices/faces/uv

In [2]:
filename = '091_W_Aya_10K.obj'
vertices = []
vertex_norm = []
vertex_tex = []
triangles = []
texcoords = []
for line in open(filename, "r"):
    values = line.split()
    if(values==[]):
        continue
    if(values=='#'):
        continue
    if(values[0]=='v'):
        vertices.append([float(values[1]),float(values[2]),float(values[3])])
    if(values[0]=='vn'):
        vertex_norm.append([float(values[1]),float(values[2]),float(values[3])])
    if(values[0]=='vt'):
        vertex_tex.append([float(values[1]),float(values[2]),float(values[3])])
    if(values[0]=='f'):
        face=[]
        texcoord = []
        norm = []
        for v in values[1:]:
            w = v.split('/')
            face.append(int(w[0]))
            if(len(w)>=2 and len(w[1])>0):
                texcoord.append(int(w[1]))
            else:
                texcoord.append(-1)
            if(len(w)>=3 and len(w[2])>0):
                norm.append(int(w[2]))
            else:
                norm.append(-1)
        triangles.append(face)
        texcoords.append(texcoord)

In [3]:
vertices = np.array(vertices,np.float32)
vertices = vertices-np.min(vertices)
vertices[...,1] = np.max(vertices[...,1]) - vertices[...,1] #顶点
triangles = np.array(triangles,np.int)-1 # 三角面片中顶点索引
texcoords=np.array(texcoords,np.int)-1 # 三角面片中纹理坐标索引
vertex_tex = np.array(vertex_tex,np.float32) # 纹理坐标

In [4]:
print(triangles.shape,texcoords.shape,vertex_tex.shape)

(10000, 3) (10000, 3) (6720, 3)


In [5]:
uv_map = cv2.imread("./tex/091_W_Aya_2K_01.jpg")
uv_map = uv_map.astype('float32')
height = uv_map.shape[0]
width = uv_map.shape[1]

In [6]:
render_width = int(np.ceil(np.max(vertices[...,0])))
render_height = int(np.ceil(np.max(vertices[...,1])))
render_img = np.zeros((render_height,render_width,3),dtype=np.uint8)
depth = np.zeros((render_height,render_width),dtype=np.float32)
depth = depth-9999
for i in range(triangles.shape[0]):
    if(i%1000==0):
        print("process:",i)
    # get uv texture map triangle
    triangle_uv = np.float32([[vertex_tex[texcoords[i][0]][0]*height,(1-vertex_tex[texcoords[i][0]][1])*width],
                             [vertex_tex[texcoords[i][1]][0]*height,(1-vertex_tex[texcoords[i][1]][1])*width],
                             [vertex_tex[texcoords[i][2]][0]*height,(1-vertex_tex[texcoords[i][2]][1])*width]])
    #get corresponding triangle in 3D face model
    triangle_3d = np.float32([[vertices[triangles[i][0]][0],vertices[triangles[i][0]][1]],
                             [vertices[triangles[i][1]][0],vertices[triangles[i][1]][1]],
                             [vertices[triangles[i][2]][0],vertices[triangles[i][2]][1]]])
    # get affine transform matrix
    warp_mat = cv2.getAffineTransform(triangle_uv,triangle_3d)
    dst = cv2.warpAffine(uv_map,warp_mat,(height,width))

    # get draw mask
    mask = np.zeros((height,width,3),dtype=np.uint8)
    cv2.drawContours(mask,[triangle_3d[np.newaxis,...].astype(np.int)],-1,(255,255,255),-1)    

    # judge depth
    mask_idx = np.argwhere(mask[...,0]==255)
    curr_depth = (vertices[triangles[i][0]][2]+vertices[triangles[i][1]][2]+vertices[triangles[i][2]][2])/3  

    # render with triangles put the mask image to final result image using opencv
    # render_img = cv2.copyTo(dst,mask,render_img)

    # render with pixel wise
    for idx in range(mask_idx.shape[0]):
        x = mask_idx[idx,0]
        y = mask_idx[idx,1]
        if(curr_depth>=depth[x,y]):
            render_img[x,y] = dst[x,y]
            depth[x,y] = curr_depth 
    
cv2.imwrite("test.jpg",render_img)

process: 0
process: 1000
process: 2000
process: 3000
process: 4000
process: 5000
process: 6000
process: 7000
process: 8000
process: 9000


True